## Import Needed Libraries

In [412]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor , BaggingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.metrics import accuracy_score , classification_report
import re
import warnings
warnings.filterwarnings('ignore')

## Reading Dataset And Get Info

### Load Data

In [413]:
df = pd.read_csv('Car details v3.csv')

In [411]:
# Show Sample of Data
df.sample(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
7101,Hyundai Accent Executive LPG,2010,165000,80000,LPG,Individual,Manual,First Owner,13.2 km/kg,1495 CC,93.7 bhp,124.5Nm@ 3500rpm,5.0
5526,Mahindra TUV 300 mHAWK100 T8 Dual Tone,2018,800000,70000,Diesel,Individual,Manual,Second Owner,18.49 kmpl,1493 CC,100 bhp,240Nm@ 1600-2800rpm,7.0
3731,Hyundai i10 Magna,2014,320000,80000,Petrol,Individual,Manual,Second Owner,20.36 kmpl,1197 CC,78.9 bhp,111.7Nm@ 4000rpm,5.0
85,Maruti Alto 800 VXI BSIV,2016,315000,32000,Petrol,Dealer,Manual,First Owner,24.7 kmpl,796 CC,47.3 bhp,69Nm@ 3500rpm,5.0
7100,Tata Tiago 1.05 Revotorq XT,2016,400000,60000,Diesel,Individual,Manual,First Owner,27.28 kmpl,1047 CC,69 bhp,140Nm@ 1800-3000rpm,5.0


In [359]:
# Show Shape of Data
print(f"Number of Row : {df.shape[0]}\nNumber of Columns : {df.shape[1]}")

Number of Row : 8128
Number of Columns : 13


In [360]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [361]:
# Check NaN Value
df.isna().sum()

name               0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          221
engine           221
max_power        215
torque           222
seats            221
dtype: int64

In [362]:
# Describe Numiric Data
df.describe()

,year,selling_price,km_driven,seats
count,8128.000000,8.128000e+03,8.128000e+03,7907.000000
mean,2013.804011,6.382718e+05,6.981951e+04,5.416719
std,4.044249,8.062534e+05,5.655055e+04,0.959588
min,1983.000000,2.999900e+04,1.000000e+00,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000
75%,2017.000000,6.750000e+05,9.800000e+04,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,14.000000


In [363]:
# Describe non Numiric 
df.describe(exclude=np.number)

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,8128,8128,8128,8128,8128,7907,7907,7913,7906
unique,2058,4,3,2,5,393,121,322,441
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm
freq,129,4402,6766,7078,5289,225,1017,377,530


In [364]:
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
name,8128,0,0.000000,2058
year,8128,0,0.000000,29
selling_price,8128,0,0.000000,677
km_driven,8128,0,0.000000,921
fuel,8128,0,0.000000,4
seller_type,8128,0,0.000000,3
transmission,8128,0,0.000000,2
owner,8128,0,0.000000,5
mileage,8128,221,2.718996,393
engine,8128,221,2.718996,121


In [365]:
df.duplicated().sum()

1202

Data Has some issues should be solved:
* Nulls values in mileage, engine, max_power, torque, seats column.
* Duplicated values 
* There are issues in data types.


## Data cleaning and preprocessing

#### Drop duplicates rows

In [366]:
df.drop_duplicates(inplace=True)

#### Handle data types issues

In [367]:
def extract_numeric(col):
    return pd.to_numeric(col.str.extract(r'(\d+\.?\d*)')[0], errors='coerce')

In [368]:
for col in ['mileage', 'engine', 'max_power']:
    df[col] = extract_numeric(df[col])

#### Clean name column

In [369]:
df[['Brand', 'Model']] = df['name'].str.extract(r'([^ ]+) (.+)', expand=True)

In [370]:
df.drop(columns= ['name'], inplace=True)

#### Clean torque column

`280Nm@ 1800-2800rpm`

280 (العزم)

1800 (أقل عدد لفات)

2800 (أعلى عدد لفات)



In [371]:
def extract_torque_info(text):
    nums = re.findall(r'\d+\.?\d*', str(text))
    torque = float(nums[0]) if len(nums) > 0 else None
    rpm_min = float(nums[1]) if len(nums) > 1 else None
    rpm_max = float(nums[2]) if len(nums) > 2 else rpm_min 
    return pd.Series([torque, rpm_min, rpm_max])

df[['torque_value', 'rpm_min', 'rpm_max']] = df['torque'].apply(extract_torque_info)

In [372]:
df.drop(columns=['torque'], inplace=True)

In [373]:
df.dtypes

year               int64
selling_price      int64
km_driven          int64
fuel              object
seller_type       object
transmission      object
owner             object
mileage          float64
engine           float64
max_power        float64
seats            float64
Brand             object
Model             object
torque_value     float64
rpm_min          float64
rpm_max          float64
dtype: object

In [374]:
df.isna().sum()

year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          208
engine           208
max_power        206
seats            208
Brand              0
Model              0
torque_value     209
rpm_min          213
rpm_max          213
dtype: int64

#### Handling Nulls

The number of missing values ​​is small and can be ignored. (Maximum 225 from 8128 Rows)

In [375]:
df.dropna(inplace=True)

In [376]:
# Data After Delete Nulls
df.sample(5)

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Brand,Model,torque_value,rpm_min,rpm_max
2288,2011,250000,35000,Petrol,Individual,Manual,First Owner,16.36,1495.0,93.7,5.0,Hyundai,Accent Executive,124.6,3500.0,3500.0
4814,2019,1100000,50000,Diesel,Individual,Manual,Second Owner,21.90,1498.0,98.6,7.0,Honda,BRV i-DTEC V MT,200.0,1750.0,1750.0
4756,2017,819999,30000,Petrol,Dealer,Manual,First Owner,17.50,1199.0,88.7,5.0,Honda,WR-V i-VTEC VX,110.0,4800.0,4800.0
5404,2010,225000,50000,Petrol,Dealer,Manual,First Owner,20.36,1197.0,78.9,5.0,Hyundai,i10 Magna,111.7,4000.0,4000.0
6138,2012,200000,30078,Petrol,Dealer,Manual,First Owner,15.30,1396.0,70.0,5.0,Tata,Indica V2 GLX BSIII,12.4,2.0,600.0


In [377]:
# The New of Shape Data
print(f"Number of Row : {df.shape[0]}\nNumber of Columns : {df.shape[1]}")

Number of Row : 6713
Number of Columns : 16


In [378]:
# Check NaN Value
df.isna().sum()

year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
mileage          0
engine           0
max_power        0
seats            0
Brand            0
Model            0
torque_value     0
rpm_min          0
rpm_max          0
dtype: int64

In [379]:
# The New Cardinality
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
year,6713,0,0.0,27
selling_price,6713,0,0.0,668
km_driven,6713,0,0.0,898
fuel,6713,0,0.0,4
seller_type,6713,0,0.0,3
transmission,6713,0,0.0,2
owner,6713,0,0.0,5
mileage,6713,0,0.0,381
engine,6713,0,0.0,121
max_power,6713,0,0.0,318


#### Drop Model column for high cardanality

In [380]:
df.drop(columns=['Model'], inplace=True)

#### Create Car Age

In [381]:
df['Car age'] = 2025 - df['year']


## EDA and Get Insights

In [382]:
top_20_brands = df['Brand'].value_counts().nlargest(20).reset_index()
top_20_brands.columns = ['Brand', 'Count']  

fig = px.bar(top_20_brands,
             x='Brand',
             y='Count',
             title='Top 20 Car Brands',
             labels={'Count': 'Number of Cars', 'Brand': 'Car Brand'},
             color='Brand'
             )

fig.update_layout(
)

fig.show()

#### Total Selling Price per Years

In [383]:
sum_Price_Years = df.groupby('year')['selling_price'].sum()
px.line(sum_Price_Years,
        x = sum_Price_Years.index,
        y = sum_Price_Years.values,
        labels={'y':'Total Selling Price','year':'Year of Selling the Car'},
        template = 'plotly_dark',
        markers = True,
        line_shape = 'spline',
        color_discrete_sequence = ['red'],
        log_x = True)

In [384]:
print(f"Best year for total car sales profit in the Market in '{sum_Price_Years.idxmax()}' is ${sum_Price_Years.values.max():,.2f}")

Best year for total car sales profit in the Market in '2017' is $586,750,970.00


## Does the type of fuel affect on the Selling Price?

In [385]:
type_of_fuel = df['fuel'].value_counts()
px.pie(title='Types of Fuel',
       values = type_of_fuel.values,
       names = type_of_fuel.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

In [386]:
price_Type_Fuel = df.groupby('fuel')['selling_price'].mean()
px.bar(x = price_Type_Fuel.index,
       y = price_Type_Fuel.values,
       template = 'plotly_dark',
       color = price_Type_Fuel.index,
       text_auto = True,
       title = 'Fuel type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Fuel'}
       )

- Diesel and Petrol fuel are the most popluar in the data
- price goes higher with Diesel fuel

## Which is better transmission ,Type of owner and Type Seller Type for the selling price?

### First -> Type of transmission on Selling Price

In [387]:
type_transmission = df['transmission'].value_counts()
px.pie(title='Types of Transmission',
       values = type_transmission.values,
       names = type_transmission.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- Manual Cars are most popular tham automatic cars

In [388]:
Selling_price_transmission = df.groupby('transmission')['selling_price'].mean()
px.bar(x = Selling_price_transmission.index,
       y = Selling_price_transmission.values,
       template = 'plotly_dark',
       color = Selling_price_transmission.index,
       text_auto = True,
       title = 'Transmission type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Transmission'}
       )

### Second -> Type of Seller on Selling Price

In [389]:
seller_types = df['seller_type'].value_counts()
px.pie(title = 'Seller type affects selling Price',
       values = seller_types.values,
       names = seller_types.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- Individual sellers are most popular than other sellers

In [390]:
Selling_price_seller_types = df.groupby('seller_type')['selling_price'].mean()
px.bar(x = Selling_price_seller_types.index,
       y = Selling_price_seller_types.values,
       template = 'plotly_dark',
       color = Selling_price_seller_types.index,
       text_auto = True,
       title = 'Seller type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Seller'}
       )

### Third -> Type of Owner on Selling Price

In [391]:
owner_types = df['owner'].value_counts()
px.pie(title = 'Seller type affects selling Price',
       values = owner_types.values,
       names = owner_types.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- First Owner are most popular than other Owners

In [392]:
Selling_price_owner_types = df.groupby('owner')['selling_price'].mean()
px.bar(x = Selling_price_owner_types.index,
       y = Selling_price_owner_types.values,
       template = 'plotly_dark',
       color = Selling_price_owner_types.index,
       text_auto = True,
       title = 'Owner type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Owner'}
       )

## Total Insights :
- Diesel and Petrol fuel are the most popluar in the data
- Individual sellers are most popular than other sellers
- First Owner are most popular than other Owners
- Manual Cars are most popular tham automatic cars
- price goes higher with Diesel fuel, automatic transmission, Dealer Seller and test Drive Car

## Premodeling

### Encode Object DataType 

In [ ]:
column_to_encode = ['Brand','fuel','seller_type','transmission','owner']
le = LabelEncoder() 
for column in column_to_encode:
    df[column] = le.fit_transform(df[column])

In [394]:
# Show Data After some of Encode
df.head()

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Brand,torque_value,rpm_min,rpm_max,Car age
0,2014,450000,145500,1,1,1,0,23.40,1248.0,74.00,5.0,20,190.0,2000.0,2000.0,11
1,2014,370000,120000,1,1,1,2,21.14,1498.0,103.52,5.0,26,250.0,1500.0,2500.0,11
2,2006,158000,140000,3,1,1,4,17.70,1497.0,78.00,5.0,10,12.7,2.0,700.0,19
3,2010,225000,127000,1,1,1,0,23.00,1396.0,90.00,5.0,11,22.4,1750.0,2750.0,15
4,2007,130000,120000,3,1,1,0,16.10,1298.0,88.20,5.0,20,11.5,4.0,500.0,18


#### Splitting Data 

In [396]:
X = df.drop('selling_price' , axis = 1)
y = df['selling_price']

In [397]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.25 , random_state = 44 , shuffle = True)

In [398]:
print(f'Shape of X_Train {X_train.shape}')
print(f'Shape of X_Test {X_test.shape}')
print(f'Shape of Y_Train {y_train.shape}')
print(f'Shape of Y_Test {y_test.shape}')

Shape of X_Train (5034, 15)
Shape of X_Test (1679, 15)
Shape of Y_Train (5034,)
Shape of Y_Test (1679,)


## Modeling

#### Use Different Models Algorithm

In [ ]:
def Kfold(model):
    model = cross_val_score(model , X , y , cv = 10) #[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    model_score = np.average(model)
    print(f"score on cross validation: {model_score * 100}%")

def train(model):
    model.fit(X_train,y_train)
    model_train_score = model.score(X_train,y_train)
    model_test_score = model.score(X_test,y_test)
    print(f"score on Training data: {model_train_score * 100}%\n score on Testing data: {model_test_score * 100}%")

def r2(model):
    score = r2_score(y_test , model.predict(X_test))
    print(f"R2 Score is {score * 100}%")

## Linear Regression Model

As base model

In [ ]:
lr = LinearRegression()   
Kfold(lr)
train(lr)
r2(lr)

score on cross validation: 61.889160003070806%
score on Training data: 63.8917598993321%
 score on Testing data: 61.129055363692665%
R2 Score is 61.129055363692665%


## Random Forest Model

In [401]:
rf_model = RandomForestRegressor(n_estimators = 100 , max_depth = 10)
Kfold(rf_model)
train(rf_model)
r2(rf_model)

score on cross validation: 88.11378643210486%
score on Training data: 97.01025391229291%
 score on Testing data: 90.11653832894935%
R2 Score is 90.11653832894935%


## KNN

In [402]:
knn = KNeighborsRegressor()
Kfold(knn)
train(knn)
r2(knn)

score on cross validation: 40.850353802297526%
score on Training data: 61.03171997956033%
 score on Testing data: 40.62578820436562%
R2 Score is 40.62578820436562%


## Decision Tree Regressor

In [403]:
DTR = DecisionTreeRegressor()
Kfold(DTR)
train(DTR)
r2(DTR)

score on cross validation: 80.8321497995818%
score on Training data: 99.92032195953901%
 score on Testing data: 84.14876109121514%
R2 Score is 84.14876109121514%


## XGBoost

In [404]:
xgboost = XGBRegressor()
Kfold(xgboost)
train(xgboost)
r2(xgboost)

score on cross validation: 88.92840467652616%
score on Training data: 99.06595187837813%
 score on Testing data: 91.45611888058784%
R2 Score is 91.45611888058784%


- Best Model is XGBoost with accuracy 97.4%

## Save rf_model Model 

In [ ]:
joblib.dump(rf_model,'rf_model.sav')

['rf_model.sav']